In [25]:
# imports
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import re

In [26]:
# load data set
df = pd.read_csv("data.csv", encoding='latin-1') 
df.columns = ['label', 'text']
print(df.head())
print(df.columns)

      label                                               text
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...
Index(['label', 'text'], dtype='object')


In [27]:
# clean text
def clean_text(text):
    text = text.lower() # convert string to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # remove links
    text = re.sub(r'\@w+|\#','', text)  # remove mentions and hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove numbers and punctuation
    return text
df['text'] = df['text'].astype(str).apply(clean_text) # updates text column to string and cleans

# encode labels
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])  

print("Label classes:", encoder.classes_)

Label classes: ['negative' 'neutral' 'positive']


In [28]:
# train/tests split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)
# here were training it so it can predict what label each text goes under
print("Unique labels:", sorted(set(train_labels)))


Unique labels: [0, 1, 2]


In [29]:
# create data set class
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from collections import Counter
import pickle


# splits text at any whitespace
def tokenizer(text):
    return text.split()

# goes through all of the text and yeilds the tokens
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


def build_vocab(texts, min_freq=2):

    counter = Counter()
    for text in texts:
        counter.update(tokenizer(text))
    # start indices at 2 so we can reserve 0 for PAD and 1 for UNK
    vocab = {word: i+2 for i, (word, freq) in enumerate(counter.items()) if freq >= min_freq}
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1
    return vocab

vocab_dict = build_vocab(train_texts, min_freq=2)  

words_to_keep = list(vocab_dict.keys())

# Create a new dictionary assigning completely new sequential indices
new_vocab_dict = {}
for i, word in enumerate(words_to_keep):
    if word not in ["<pad>", "<unk>"]:
        new_vocab_dict[word] = i + 2

new_vocab_dict["<pad>"] = 0
new_vocab_dict["<unk>"] = 1
vocab_dict = new_vocab_dict
final_length = len(vocab_dict)
final_max_index = max(vocab_dict.values())


def numericalize(text, vocab):
    tokens = tokenizer(text)  # split into words
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]

with open("vocab.pkl", "wb") as f:
    pickle.dump(vocab_dict, f)


In [30]:

class NewsDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab

    # number of samples in dataset
    def __len__(self):
        return len(self.texts)

    # call when we want only one sample
    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        tokens = torch.tensor(numericalize(text, self.vocab), dtype=torch.long) #self.vocab is for mapping words to numbers
        return tokens, torch.tensor(label, dtype=torch.long) #returns tokens and labels
    
# make data into dataset object
train_dataset = NewsDataset(train_texts, train_labels, vocab_dict)
test_dataset = NewsDataset(test_texts, test_labels, vocab_dict)

def collate_batch(batch):
    texts, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=0) #makes all lists same length by adding zeros at end
    labels = torch.tensor(labels, dtype=torch.long)
    return texts, labels

# train 32 healines per batch
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch) # reshuffle every epoch
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_batch)

print("Train batches:", len(train_loader))
print("Test batches:", len(test_loader))

Train batches: 122
Test batches: 31


In [31]:
import torch.nn as nn

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  #turns vocab id into dense vector
        #self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.fc = nn.Linear(embed_dim, num_classes) # input is average embedding of a sentence and output is num_classes(2)cwhich is buy or sell
    
    def forward(self, x):
        # x = (batch size, seq len)
        embedded = self.embedding(x) # gives us original output and embed_dim
        pooling = embedded.mean(dim = 1) # average embedding across all words in teh sentence, used to get one vec per sentence
        output = self.fc(pooling) # passes vector into classifier
        return output

In [32]:
#initialize model, loss, optimizer
vocab_size = final_max_index + 1 # amount of unique words
embed_dim = 50 # size of each word vector, the larger the number the more expressive the word is
num_classes = len(set(train_labels)) # number of classes -> 2(buy/sell)

model = TextClassifier(vocab_size, embed_dim, num_classes)

criterion = nn.CrossEntropyLoss() # diff between prediction and target
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # optimizer that updates weights using gradients


In [33]:
# model training loop
def train_model(self, train_loader, criterion, optimizer, epochs = 5):
    model.train() # put in training mode
    for epoch in range(epochs):
        total_loss = 0
        for texts, label in train_loader:
            optimizer.zero_grad() # reset gradients
            outputs = model(texts) # forward pass
            loss = criterion(outputs, label) # calculate error
            loss.backward() # back propagation
            optimizer.step() # update weights
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

In [34]:
def evaluate_model(self, test_loader):
    model.eval() # put in evaluation mode
    correct, total = 0, 0
    with torch.no_grad(): # gradients not needed in eval mode
        for texts, labels in test_loader:
            outputs = model(texts) # forward pass
            _, predicted = torch.max(outputs, 1) # get prediction index 0 = sell 1 = buy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [35]:
# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=30)

torch.save(model.state_dict(), "sentiment_model.pth")


# Evaluate on test set
evaluate_model(model, test_loader)


Epoch 1/30, Loss: 115.0568
Epoch 2/30, Loss: 109.9712
Epoch 3/30, Loss: 105.8915
Epoch 4/30, Loss: 102.7013
Epoch 5/30, Loss: 98.5057
Epoch 6/30, Loss: 94.2129
Epoch 7/30, Loss: 90.7214
Epoch 8/30, Loss: 87.1971
Epoch 9/30, Loss: 83.9083
Epoch 10/30, Loss: 81.2496
Epoch 11/30, Loss: 78.0844
Epoch 12/30, Loss: 74.4518
Epoch 13/30, Loss: 71.4954
Epoch 14/30, Loss: 68.3382
Epoch 15/30, Loss: 65.4013
Epoch 16/30, Loss: 62.4374
Epoch 17/30, Loss: 59.6437
Epoch 18/30, Loss: 56.9152
Epoch 19/30, Loss: 54.2372
Epoch 20/30, Loss: 52.6700
Epoch 21/30, Loss: 49.8093
Epoch 22/30, Loss: 47.4438
Epoch 23/30, Loss: 45.2833
Epoch 24/30, Loss: 42.9911
Epoch 25/30, Loss: 42.0814
Epoch 26/30, Loss: 39.3460
Epoch 27/30, Loss: 37.7534
Epoch 28/30, Loss: 36.4365
Epoch 29/30, Loss: 34.6813
Epoch 30/30, Loss: 33.1965
Test Accuracy: 77.22%


In [36]:
checkpoint = torch.load("sentiment_model.pth", map_location="cpu")
for key, value in checkpoint.items():
    print(key, value.shape)

embedding.weight torch.Size([4039, 50])
fc.weight torch.Size([3, 50])
fc.bias torch.Size([3])


In [37]:
# use model
import torch.nn.functional as F
def prediction(text, model, vocab, max_len = 50):
    model.eval()
    with torch.no_grad():
        # tokenize
        tokens = text.lower().split()
        token_ids = [vocab_dict.get(word, vocab["<unk>"]) for word in tokens]

        # pad
        if len(tokens) < max_len:
            token_ids += [vocab["<pad>"]] * (max_len - len(token_ids))
        else:
            token_ids = token_ids[:max_len]

        input_tensor = torch.tensor([token_ids])
        output = model(input_tensor)

        probs = F.softmax(output, dim=1) # rescales input and has it sum to 1, probability distribution
        predicted_class = torch.argmax(probs, dim=1).item() # picks class with highest probability
        return predicted_class, probs.numpy()

In [38]:
sample_text = "Stock prices soar after company reports record earnings"
pred_class, pred_probs = prediction(sample_text, model, vocab_dict)

print("Prediction:", pred_class)
print("Probabilities:", pred_probs)
example_texts = [
    "With the new production plant the company would increase its capacity to meet the expected increase",
    "Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing",
    "The international electronic industry company Elcoteq has laid off tens of employees from its Tallin"
]

for txt in example_texts:
    pred_class, probs = prediction(txt, model, vocab_dict)
    print(f"Text: {txt}")
    print(f"Predicted Class: {pred_class}, Probabilities: {probs}\n")


Prediction: 2
Probabilities: [[0.1959698 0.2850381 0.5189921]]
Text: With the new production plant the company would increase its capacity to meet the expected increase
Predicted Class: 2, Probabilities: [[0.00738128 0.12397449 0.86864424]]

Text: Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing
Predicted Class: 2, Probabilities: [[0.1801121  0.03482138 0.7850665 ]]

Text: The international electronic industry company Elcoteq has laid off tens of employees from its Tallin
Predicted Class: 0, Probabilities: [[0.5691825  0.09639616 0.3344213 ]]



In [39]:
# If you used sklearn LabelEncoder earlier and saved it as `encoder`:
print("Label encoder classes (index -> label):")
for i, lbl in enumerate(encoder.classes_):
    print(i, lbl)

# If you used manual mapping like {"negative":0, "neutral":1, "positive":2}:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
print(label_map)

Label encoder classes (index -> label):
0 negative
1 neutral
2 positive
{0: 'negative', 1: 'neutral', 2: 'positive'}
